In [1]:
import numpy as np
import sys
from sklearn.utils import class_weight
from keras import backend as K
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Flatten, Dense, Input, Conv1D, MaxPooling1D
from keras.regularizers import l1_l2
from keras.utils import to_categorical
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:


AES_Sbox = np.array(
    [0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76, 0xCA, 0x82, 0xC9,
     0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0, 0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F,
     0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15, 0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07,
     0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75, 0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3,
     0x29, 0xE3, 0x2F, 0x84, 0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58,
     0xCF, 0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8, 0x51, 0xA3,
     0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2, 0xCD, 0x0C, 0x13, 0xEC, 0x5F,
     0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73, 0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88,
     0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB, 0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC,
     0x62, 0x91, 0x95, 0xE4, 0x79, 0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A,
     0xAE, 0x08, 0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A, 0x70,
     0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E, 0xE1, 0xF8, 0x98, 0x11,
     0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF, 0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42,
     0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16])
AES_inv_Sbox = np.array(
    [0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB, 0x7C, 0xE3, 0x39,
     0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB, 0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2,
     0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E, 0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76,
     0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25, 0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC,
     0x5D, 0x65, 0xB6, 0x92, 0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D,
     0x84, 0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06, 0xD0, 0x2C,
     0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B, 0x3A, 0x91, 0x11, 0x41, 0x4F,
     0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73, 0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85,
     0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E, 0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62,
     0x0E, 0xAA, 0x18, 0xBE, 0x1B, 0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD,
     0x5A, 0xF4, 0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F, 0x60,
     0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF, 0xA0, 0xE0, 0x3B, 0x4D,
     0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61, 0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6,
     0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D])

hw = np.array([bin(x).count("1") for x in range(256)])


def load_traces(database_file, start_at=0, number_samples=0):
    traces = np.loadtxt(database_file, delimiter=',', dtype=np.float64, skiprows=1,
                        usecols=range(start_at, number_samples))
    inputoutput = np.loadtxt(database_file, delimiter=',', dtype=np.str, skiprows=1,
                             usecols=start_at - 1)
    # print("traces shape: {}\ninputoutput shape: {}\n".format(traces.shape, inputoutput.shape))
    return traces, inputoutput


def shorten_traces(dataset, start_at=0, number_samples=0):
    if len(dataset) == 2:
        traces, inputoutput = dataset
    elif len(dataset) == 3:
        traces, inputoutput, labels = dataset

    traces_selected = traces[start_at:start_at + number_samples]

    if len(dataset) == 2:
        return traces_selected, inputoutput
    elif len(dataset) == 3:
        return traces_selected, inputoutput, labels


def statcorrect_traces(dataset):
    if len(dataset) == 2:
        traces, inputoutput = dataset
    elif len(dataset) == 3:
        traces, inputoutput, labels = dataset

    # traces_statcorrect = (traces - np.mean(traces, axis=1).reshape(-1,1))/np.std(traces, axis=1).reshape(-1,1)
    traces_statcorrect = (traces - np.mean(traces, axis=0).reshape(1, -1)) / np.std(traces, axis=0).reshape(1, -1)

    if len(dataset) == 2:
        return traces_statcorrect, inputoutput
    elif len(dataset) == 3:
        return traces_statcorrect, inputoutput, labels


# noinspection PyShadowingNames
def split_data_percentage(dataset, training_fraction=0.5):
    if len(dataset) == 2:
        traces, inputoutput = dataset
    elif len(dataset) == 3:
        traces, inputoutput, labels = dataset

    traces_train = traces[:int(traces.shape[0] * training_fraction)]
    traces_test = traces[int(traces.shape[0] * training_fraction):]
    inputoutput_train = inputoutput[:int(inputoutput.shape[0] * training_fraction)]
    inputoutput_test = inputoutput[int(inputoutput.shape[0] * training_fraction):]
    if len(dataset) == 3:
        labels_train = labels[:int(labels.shape[0] * training_fraction)]
        labels_test = labels[int(labels.shape[0] * training_fraction):]

    if len(dataset) == 2:
        return (traces_train, traces_test), (inputoutput_train, inputoutput_test)
    elif len(dataset) == 3:
        return (traces_train, traces_test), (inputoutput_train, inputoutput_test), (labels_train, labels_test)


def create_labels_sboxinputkey(dataset, input_index=0, static_key=0):
    if len(dataset) == 2:
        traces, inputoutput = dataset
    elif len(dataset) == 3:
        traces, inputoutput, _ = dataset

    labels = np.zeros(inputoutput.shape)
    for i, v in enumerate(inputoutput):
        labels[i] = hw[AES_Sbox[bytes.fromhex(v)[input_index] ^ static_key]]

    return traces, inputoutput, labels

#use for hamming weight leakage model
def create_model(classes=9, number_samples=200):
    input_shape = (number_samples, 1)
    trace_input = Input(shape=input_shape)
    x = Conv1D(filters=10, kernel_size=10, strides=10, activation='relu', padding='valid', name='block1_conv1')(
        trace_input)
    x = MaxPooling1D(pool_size=1, strides=1, padding='valid', name='block1_pool')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(50, activation='tanh', name='fc1')(x)
    x = Dense(50, activation='tanh', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    model = Model(trace_input, x, name='cnn')
    optimizer = SGD(lr=0.01, decay=0, momentum=0, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

#use for bit leakage model
def create_big_model(classes=256, number_samples=200):
    input_shape = (number_samples, 1)
    trace_input = Input(shape=input_shape)
    # Block 1
    x = Conv1D(filters=64, kernel_size=10, strides=10, activation='relu', padding='same', name='block1_conv1')(
        trace_input)
    x = MaxPooling1D(pool_size=2, strides=2, padding='same', name='block1_pool')(x)
    # Block 2
    x = Conv1D(filters=128, kernel_size=10, strides=10, activation='relu', padding='same', name='block2_conv1')(
        x)
    x = MaxPooling1D(pool_size=2, strides=2, padding='same', name='block2_pool')(x)
    # Block 3
    x = Conv1D(filters=256, kernel_size=10, strides=10, activation='relu', padding='same', name='block3_conv1')(
        x)
    x = MaxPooling1D(pool_size=2, strides=2, padding='same', name='block3_pool')(x)
    # Block 4
    x = Conv1D(filters=512, kernel_size=10, strides=10, activation='relu', padding='same', name='block4_conv1')(
        x)
    x = MaxPooling1D(pool_size=2, strides=2, padding='same', name='block4_pool')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(50, activation='tanh', name='fc1')(x)
    x = Dense(50, activation='tanh', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    model = Model(trace_input, x, name='cnn')
    optimizer = SGD(lr=0.01, decay=0, momentum=0, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


if __name__ == '__main__':

    if 'dataset' not in locals():
        dataset = load_traces('../traceSWAES.csv', 1, 326)

        dataset = statcorrect_traces(dataset)
        # dataset = shorten_traces(dataset, 228405-100, 200)
        
        
        # Args: Dataset, Byte to Attack, Subkey[Bayte] of first round
        test_acc = np.zeros(16)
        key = 'DEADBEEF01234567CAFEBABE89ABCDEF'
        for i in range(16):
            rk = key[i*2:i*2+2]
            dataset_keyhype = create_labels_sboxinputkey(dataset, i, int(rk,16)) #Template - Use known SubKey byte
            dataset_keyhype = split_data_percentage(dataset_keyhype, training_fraction=0.85)
            (traces_train, traces_test), (inputoutput_train, inputoutput_test), (labels_train, labels_test) = dataset_keyhype
            
            print("<------------------Using byte: {} with subkey: {}------------------>".format(i,rk))
            # print(traces_train.shape, traces_train.dtype)
            # print(traces_test.shape, traces_test.dtype)
            # print(inputoutput_train.shape, inputoutput_train.dtype)
            # print(inputoutput_test.shape, inputoutput_test.dtype)
            # print(labels_train.shape, labels_train.dtype)
            # print(labels_test.shape, labels_test.dtype)
            # print(labels_train[0])

            min_class_tr = int(np.min(labels_train))
            min_class_ts = int(np.min(labels_test))
            classes = max(len(np.unique(labels_train)) + min_class_tr, len(np.unique(labels_test)) + min_class_ts)
            classes = 9

            traces_train_reshaped = traces_train.reshape((traces_train.shape[0], traces_train.shape[1], 1))
            labels_train_categorical = to_categorical(labels_train, num_classes=classes)
            traces_test_reshaped = traces_test.reshape((traces_test.shape[0], traces_test.shape[1], 1))
            labels_test_categorical = to_categorical(labels_test, num_classes=classes)

            save_model = ModelCheckpoint('model_epoch{epoch}.h5', period=100)


            class CalculateRecall(Callback):
                def __init__(self, data, labels, message_prefix=None):
                    self.data = data
                    self.labels = labels
                    self.message_prefix = message_prefix + ' ' or ''

                def on_epoch_end(self, epoch, logs=None):
                    logs = logs or {}

                    predictions = self.model.predict(self.data)
                    correctly_classified = (np.argmax(predictions, axis=1) == self.labels)
                    _sum = 0.
                    for i in np.unique(self.labels):
                        n_correct = len(np.nonzero(correctly_classified[np.where(self.labels == i)[0]])[0])
                        n_total = len(np.where(self.labels == i)[0])
                        _sum += n_correct / n_total
                    recall = _sum / len(np.unique(self.labels))

                    print(self.message_prefix + 'recall:', recall)


            calculate_recall_train = CalculateRecall(traces_train_reshaped, labels_train, 'train')
            calculate_recall_test = CalculateRecall(traces_test_reshaped, labels_test, 'test')
            callbacks = [calculate_recall_train, calculate_recall_test, save_model]

            model = create_model(classes=classes, number_samples=traces_train.shape[1])

            history = model.fit(x=traces_train_reshaped,
                                y=labels_train_categorical,
                                batch_size=100,
                                verbose=0,
                                epochs=250,
                                class_weight=class_weight.compute_class_weight('balanced', np.unique(labels_train),
                                                                               labels_train),
                                validation_data=(traces_test_reshaped, labels_test_categorical),
                                callbacks=callbacks)

            t = model.evaluate(x=traces_test_reshaped,
                               y=labels_test_categorical,
                               verbose=0)

            test_acc[i]=t[1]

<------------------Using byte: 0 with subkey: DE------------------>
train recall: 0.0883057774356365
test recall: 0.07532051282051282
train recall: 0.08344950053221976
test recall: 0.08814102564102565
train recall: 0.07040653402112503
test recall: 0.08012820512820513
train recall: 0.08023468844673709
test recall: 0.11675824175824175
train recall: 0.10212126678539261
test recall: 0.14239926739926742
train recall: 0.11576291656431673
test recall: 0.14239926739926742
train recall: 0.13416863690329978
test recall: 0.13438644688644688
train recall: 0.13146148059444854
test recall: 0.1472069597069597
train recall: 0.14298448630557603
test recall: 0.16483516483516483
train recall: 0.13604004186113158
test recall: 0.16483516483516483
train recall: 0.13393163278883158
test recall: 0.16483516483516483
train recall: 0.1366490240931794
test recall: 0.16483516483516483
train recall: 0.14377002220994023
test recall: 0.16483516483516483
train recall: 0.13113332105133874
test recall: 0.177655677655677

train recall: 0.3656008015463616
test recall: 0.1787087912087912
train recall: 0.3576858602075072
test recall: 0.21236263736263736
train recall: 0.3587186352218362
test recall: 0.19954212454212453
train recall: 0.36609710744405843
test recall: 0.18672161172161172
train recall: 0.36770921061866163
test recall: 0.22335164835164836
train recall: 0.372370291536536
test recall: 0.21053113553113553
train recall: 0.359195522043256
test recall: 0.19954212454212453
train recall: 0.3613141661110526
test recall: 0.21236263736263736
train recall: 0.35832906570574685
test recall: 0.18855311355311355
train recall: 0.36315486608239467
test recall: 0.18855311355311355
train recall: 0.38192913764314373
test recall: 0.22335164835164836
train recall: 0.36889806931431374
test recall: 0.22335164835164836
train recall: 0.3750876828408839
test recall: 0.22335164835164836
train recall: 0.3766059804540829
test recall: 0.2441849816849817
train recall: 0.372370291536536
test recall: 0.22037545787545787
train rec

train recall: 0.13184269434269433
test recall: 0.14523809523809522
train recall: 0.14064407814407814
test recall: 0.14523809523809522
train recall: 0.1378968253968254
test recall: 0.14523809523809522
train recall: 0.14749575043692692
test recall: 0.14523809523809522
train recall: 0.1472830927242692
test recall: 0.13095238095238096
train recall: 0.1559827264239029
test recall: 0.13095238095238096
train recall: 0.15502525796643443
test recall: 0.16666666666666669
train recall: 0.14454499748617397
test recall: 0.13095238095238096
train recall: 0.16073954368072016
test recall: 0.13095238095238096
train recall: 0.1632832962244727
test recall: 0.16666666666666669
train recall: 0.1655981110392875
test recall: 0.16666666666666669
train recall: 0.1684725514137279
test recall: 0.16666666666666669
train recall: 0.1632832962244727
test recall: 0.16666666666666669
train recall: 0.15402403696521344
test recall: 0.16666666666666669
train recall: 0.16292717086834735
test recall: 0.13095238095238096
tr

train recall: 0.450680289209701
test recall: 0.2357142857142857
train recall: 0.47594585697526876
test recall: 0.17857142857142858
train recall: 0.475716919246331
test recall: 0.17857142857142858
train recall: 0.4869094304388423
test recall: 0.10714285714285714
train recall: 0.4928618113912232
test recall: 0.12142857142857144
train recall: 0.4624882089587973
test recall: 0.12142857142857144
train recall: 0.5064951279657162
test recall: 0.12142857142857144
train recall: 0.4494804759510642
test recall: 0.12142857142857144
train recall: 0.46763389116330295
test recall: 0.12142857142857144
train recall: 0.4623774809068927
test recall: 0.12142857142857144
train recall: 0.45642509995451175
test recall: 0.17142857142857143
train recall: 0.5049049534343651
test recall: 0.1357142857142857
train recall: 0.4612845052550935
test recall: 0.1357142857142857
train recall: 0.452652206181618
test recall: 0.1357142857142857
train recall: 0.45014304867246047
test recall: 0.12142857142857144
train recall:

train recall: 0.19687067340212713
test recall: 0.15959595959595957
train recall: 0.20089773757609775
test recall: 0.15959595959595957
train recall: 0.20681465250492925
test recall: 0.1466089466089466
train recall: 0.20780770116431357
test recall: 0.1466089466089466
train recall: 0.22207615531856711
test recall: 0.1466089466089466
train recall: 0.21971766475252935
test recall: 0.1466089466089466
train recall: 0.21578022681807058
test recall: 0.15959595959595957
train recall: 0.22000068362045389
test recall: 0.1466089466089466
train recall: 0.21325622814213546
test recall: 0.15959595959595957
train recall: 0.21142214642430524
test recall: 0.15959595959595957
train recall: 0.21397402961875567
test recall: 0.15959595959595957
train recall: 0.21259419570254595
test recall: 0.15959595959595957
train recall: 0.21806258365355552
test recall: 0.13073593073593076
train recall: 0.21620061741720997
test recall: 0.1466089466089466
train recall: 0.20615262006533974
test recall: 0.15930735930735931
t

...

train recall: 0.27785379965660795
test recall: 0.2238095238095238
train recall: 0.2930595212895234
test recall: 0.20714285714285716
train recall: 0.30160806265729
test recall: 0.2113095238095238
train recall: 0.28600054096008654
test recall: 0.24464285714285713
train recall: 0.28775576044092166
test recall: 0.24464285714285713
train recall: 0.2839513606322177
test recall: 0.24464285714285713
train recall: 0.29140234102437457
test recall: 0.24464285714285713
train recall: 0.30141206262593
test recall: 0.2113095238095238
train recall: 0.30200986272157804
test recall: 0.22797619047619047
train recall: 0.3034112629458021
test recall: 0.21547619047619046
train recall: 0.30241166278586606
test recall: 0.21547619047619046
train recall: 0.30321526291444206
test recall: 0.23928571428571424
train recall: 0.315862243337959
test recall: 0.23928571428571424
train recall: 0.3009602825536452
test recall: 0.21547619047619046
train recall: 0.2975097020015523
test recall: 0.21547619047619046
train recal

train recall: 0.45143061449556504
test recall: 0.27559523809523806
train recall: 0.468122303832902
test recall: 0.27559523809523806
train recall: 0.4490014041068913
test recall: 0.27559523809523806
train recall: 0.5129753943427298
test recall: 0.27559523809523806
train recall: 0.49029557738062574
test recall: 0.27559523809523806
train recall: 0.5079753943427298
test recall: 0.27559523809523806
train recall: 0.5136231744463746
test recall: 0.27559523809523806
train recall: 0.48678346588535454
test recall: 0.27559523809523806
train recall: 0.4896843055494889
test recall: 0.27559523809523806
train recall: 0.5078056051155635
test recall: 0.27559523809523806
train recall: 0.5117699157498532
test recall: 0.27559523809523806
train recall: 0.48678346588535454
test recall: 0.27559523809523806
train recall: 0.49178346588535454
test recall: 0.27559523809523806
train recall: 0.4973812659810026
test recall: 0.27559523809523806
train recall: 0.48054155741998256
test recall: 0.3089285714285714
train 

train recall: 0.3174213630588296
test recall: 0.1693121693121693
train recall: 0.33147979740463235
test recall: 0.17460317460317457
train recall: 0.3170220519928718
test recall: 0.17460317460317457
train recall: 0.33014121774848526
test recall: 0.15608465608465608
train recall: 0.35145875021694045
test recall: 0.15608465608465608
train recall: 0.3696778114517279
test recall: 0.17460317460317457
train recall: 0.3789285052537631
test recall: 0.16269841269841268
train recall: 0.363954305678132
test recall: 0.17460317460317457
train recall: 0.3263644462707031
test recall: 0.17460317460317457
train recall: 0.3263644462707031
test recall: 0.167989417989418
train recall: 0.35244766298535424
test recall: 0.15608465608465608
train recall: 0.3669957109435404
test recall: 0.15608465608465608
train recall: 0.3373296381615612
test recall: 0.167989417989418
train recall: 0.35122273083475586
test recall: 0.15608465608465608
train recall: 0.34423965840302484
test recall: 0.167989417989418
train recall

train recall: 0.6212311201333997
test recall: 0.291005291005291
train recall: 0.6209102330589092
test recall: 0.291005291005291
train recall: 0.6193414518058452
test recall: 0.291005291005291
train recall: 0.6241609125033332
test recall: 0.2923280423280423
train recall: 0.6319408738806207
test recall: 0.3161375661375661
train recall: 0.6191342122369035
test recall: 0.30952380952380953
train recall: 0.6281659939904355
test recall: 0.3280423280423281
train recall: 0.6249943275440234
test recall: 0.3280423280423281
train recall: 0.641387134772036
test recall: 0.291005291005291
train recall: 0.623776611108063
test recall: 0.2976190476190476
train recall: 0.6367447359008764
test recall: 0.291005291005291
train recall: 0.6660661398924315
test recall: 0.2976190476190476
train recall: 0.6698293473030553
test recall: 0.2976190476190476
train recall: 0.6592999925732889
test recall: 0.28571428571428564
train recall: 0.6592999925732889
test recall: 0.28571428571428564
train recall: 0.6592999925732

train recall: 0.2754025044722719
test recall: 0.10952380952380951
train recall: 0.2851832760262993
test recall: 0.10952380952380951
train recall: 0.27582860453499985
test recall: 0.10952380952380951
train recall: 0.2873020148456195
test recall: 0.13333333333333333
train recall: 0.28439503810143346
test recall: 0.13333333333333333
train recall: 0.2885039379225426
test recall: 0.13333333333333333
train recall: 0.2902792701484562
test recall: 0.13333333333333333
train recall: 0.2873327980159376
test recall: 0.13333333333333333
train recall: 0.28951862091396974
test recall: 0.13333333333333333
train recall: 0.2706068640661664
test recall: 0.15238095238095237
train recall: 0.2706068640661664
test recall: 0.15238095238095237
train recall: 0.2585429105777943
test recall: 0.1619047619047619
train recall: 0.2973415392746788
test recall: 0.14285714285714285
train recall: 0.2979768051715726
test recall: 0.13333333333333333
train recall: 0.28693944137258093
test recall: 0.12857142857142856
train r

In [3]:
test_acc

array([0.37837838, 0.27027027, 0.21621622, 0.37837838, 0.18918919,
       0.27027027, 0.27027027, 0.18918919, 0.13513514, 0.37837838,
       0.27027027, 0.32432432, 0.21621622, 0.35135135, 0.35135135,
       0.32432433])